# End-to-End MCP Application

## Buliding the Gradio MCP Server

In this example, we will create a sentiment analysis MCP server using Gradio. This server will expose a sentiment analysis tool that can be used by both human users through a web interface and AI models through the MCP protocol.

### Setting up

```bash
mkdir mcp-sentiment
cd mcp-sentiment
python -m venv venv
source venv/bin/activate  # On Windows: venv\Scripts\activate
pip install "gradio[mcp]" textblob
```

### Creating the Server

HuggingFace Spaces need an `app.py` file to build the space, so the name of the Python file has to be `app.py`.

In [ ]:
# app.py

import gradio as gr
from textblob import TextBlob

def sentiment_analysis(text: str) -> dict:
    """Analyze the sentiment of the given text.

    Args:
        text (str): The text to analyze

    Returns:
        dict: A dictionary containing polarity, subjectivity, and assessment
    """
    blob = TextBlob(text)
    sentiment = blob.sentiment

    return {
        'polarity': round(sentiment.polarity, 2), # -1 (negative) to 1 (positive)
        'subjectivity': round(sentiment.subjectivity, 2), # 0 (objective) to 1 (subjective)
        'assessment': 'positive' if sentiment.polarity > 0 else 'negative' if sentiment.polarity < 0 else 'neutral'
    }



# Create the Gradio interface
demo = gr.Interface(
    fn=sentiment_analysis,
    inputs=gr.Textbox(placeholder="Enter text to analyze..."),
    outputs=gr.JSON(),
    title="Text Sentiment Analysis",
    description="Analyze the sentiment of text using TextBlob"
)

# Launch the interface and MCP server
if __name__ == "__main__":
    demo.launch(mcp_server=True)

- Function definition
    - The `setiment_analysis` takes a text input and return a dictionary
    - It uses `TextBlob` to analyze the sentiment
    - The docstring is crucial as it helps Gradio generate the MCP tool schema
    - Type hints (`str` and `dict`) help define the input/output schema

- Gradio interface
    - `gr.Interface` creates both the web UI and MCP server
    - The function is exposed as an MCP tool automatically
    - Input and output components define the tool's schema
    - The JSON output component ensures proper serlaization

- MCP server
    - Setting `mcp_server=True` enables the MCP server
    - The server will be available at `http://localhost:7860/gradio_api/mcp/sse`
    - We can also enable it using the environment variable:
    ```bash
    export GRADIO_MCP_SERVER=True
    ```



To start the server, we just need to run
```bash
python app.py
```

### Troubleshooting

- Type hints and docstrings
    - Always provide type hints for our function parameters and return values
    - Include a docstring with a "Args: " block for each parameter
    - This helps Gradio generate accurate MCP tool schema
- String inputs
    - When in doubt, accept input arguments as `str`
    - Convert them to the desired type inside the function
    - This provides better compatibility with MCP clients
- SSE support
    - Some MCP clients do not support SSE-based MCP servers
    - In those cases, use `mcp-remote`:
    ```json
    {
        "mcpServers": {
            "gradio": {
            "command": "npx",
            "args": [
                "mcp-remote",
                "http://localhost:7860/gradio_api/mcp/sse"
            ]
            }
        }
    }
    ```
- Connection isues
    - If we encounter connection issues, try restarting both the client and server
    - Check that the server is running and accessible
    - Verify that the MCP schema is available at the expected URL

## Using MCP Clients with Our Application

In this example, we will create MCP clients that can interact with our MCP server using different programming languages.

MCP hosts use configuration files to manage server connections. These files define which servers are available and how to connect them.

The standard configuration file for MCP is named `mcp.json`. Here is a basic structure:
```json
{
  "servers": [
    {
      "name": "MCP Server",
      "transport": {
        "type": "sse",
        "url": "http://localhost:7860/gradio_api/mcp/sse"
      }
    }
  ]
}
```
Now we have a single server configured to use SSE transport, connecting to a local Gradio server running on port 7860. Here we connected to the Gradio app via SSE transport because we assume that the gradio app is running on a remote server. If want to connect to a local script, we need to switch to `stdio` transport.


For example, for remote servers using HTTP+SSE transport, the configuration includes the server URL:
```json
{
  "servers": [
    {
      "name": "Remote MCP Server",
      "transport": {
        "type": "sse",
        "url": "https://example.com/gradio_api/mcp/sse"
      }
    }
  ]
}
```

When working with Gradio MCP servers, we can configure our UI client to connect to the server using the MCP protocol by creating a new file called `config.json`:
```json
{
  "mcpServers": {
    "mcp": {
      "url": "http://localhost:7860/gradio_api/mcp/sse"
    }
  }
}
```

## Building an MCP Client with Gradio

In this example, we will use Gradio as an MCP Client to connect to an MCP Server.

In [ ]:
!pip install "smolagents[mcp]" "gradio[mcp]" mcp

In [ ]:
import gradio as gr

from mcp.client.stdio import StdioServerParameters
from smolagents import ToolCollection, CodeAgent, InferenceClientModel
from smolagents.mcp_client import MCPClient

Next, we will connect to the MCP Server and get the tools that we can use to answer questions.

In [ ]:
mcp_client = MCPClient(
    {'url': "http://localhost:7860/gradio_api/mcp/sse"}
)

tools = mcp_client.get_tools()

Now that we have the tools, we can create a simple agent that uses them to answer questions.

In [ ]:
model = InferenceClientModel()
agent = CodeAgent(model=model, tools=[*tools])

Now we can create a simple Gradio interface that uses the agent to answer questions.

In [ ]:
demo = gr.ChatInterface(
    fn=lambda message, history: str(agent.run(message)),
    type="messages",
    examples=['Prime factorization of 68'],
    title='Agent with MCP Tools',
    description="This is a simple agent that uses MCP tools to answer questions.",
    messages=[]
)

demo.launch()

### Complete Snippet

In [ ]:
# app.py

import gradio as gr

from mcp.client.stdio import StdioServerParameters
from smolagents import ToolCollection, CodeAgent
from smolagents import CodeAgent, InferenceClientModel
from smolagents.mcp_client import MCPClient


try:
    mcp_client = MCPClient(
        {"url": "http://localhost:7860/gradio_api/mcp/sse"}
    )
    tools = mcp_client.get_tools()

    model = InferenceClientModel()
    agent = CodeAgent(tools=[*tools], model=model)

    demo = gr.ChatInterface(
        fn=lambda message, history: str(agent.run(message)),
        type="messages",
        examples=["Prime factorization of 68"],
        title="Agent with MCP Tools",
        description="This is a simple agent that uses MCP tools to answer questions.",
    )

    demo.launch()
finally:
    mcp_client.close()

It is important to have the `finally` block because the MCP Client is a long-lived object that needs to be closed when the program exits.

## Building a Tiny Agent with TypeScript

In this example, we will implement a TypeScript (JS) MCP client that can coomunicate with any MCP server, including the Gradio-based sentiment analysis server we had in previous section.

If we have NodeJS (with `pnpm` or `npm`), we can run
```bash
npx @huggingface/mcp-client
```
or if using `pnpm`:
```bash
pnpx @huggingface/mcp-client
```
This installs the package into a temporary folder then executes its command.

We will see a simple agent connecting to multiple MCP servers (running locally), loading their tools, then prompting us for a conversation. By default, our example agent connects to two MCP servers:
- the "canonical" [file system server](https://github.com/modelcontextprotocol/servers/tree/main/src/filesystem) to get access to our Desktop, and
- the [Playwright MCP server](https://github.com/microsoft/playwright-mcp) to use a sandboxed Chromium browser for us.


### Default model and provider

Our example agent uses by default:
- `Qwen/Qwen2.5-72B-Instruct`
- running on [Nebius](https://huggingface.co/docs/inference-providers/providers/nebius)


This is all configurable through `env` variable. We can add our Gradio MCP server:
```typescript
const agent = new Agent({
	provider: process.env.PROVIDER ?? "nebius",
	model: process.env.MODEL_ID ?? "Qwen/Qwen2.5-72B-Instruct",
	apiKey: process.env.HF_TOKEN,
	servers: [
		// Default servers
		{
			command: "npx",
			args: ["@modelcontextprotocol/servers", "filesystem"]
		},
		{
			command: "npx",
			args: ["playwright-mcp"]
		},
		// Our Gradio sentiment analysis server
		{
			command: "npx",
			args: [
				"mcp-remote",
				"http://localhost:7860/gradio_api/mcp/sse"
			]
		}
	],
});
```
We connect our Gradio-based MCP server via the [mcp-remote](https://www.npmjs.com/package/mcp-remote) pacakge.

### Tool calling native support in LLMs

A tool is defined by its name, a description, and a JSONSchema representation of its parameters - exactly how we defined our sentiment analysis function in the Gradio server. For example,
```typescript
const weatherTool = {
	type: "function",
	function: {
		name: "get_weather",
		description: "Get current temperature for a given location.",
		parameters: {
			type: "object",
			properties: {
				location: {
					type: "string",
					description: "City and country e.g. Bogotá, Colombia",
				},
			},
		},
	},
};
```

Inference engines let us pass a list of tools when calling the LLM, and the LLM is free to call zero, one, or more tools.

In the backend (at the inference engine level), the tools are simply passed to the model in a specially-formatted `chat_template`, liek any other message, and then parsed out of the response (using model-specific special tokens) to expose them as tool calls.

### Implementing an MCP client on top of InferenceClient

The complete `McpClient.ts` code file is [HERE](https://github.com/huggingface/huggingface.js/blob/main/packages/mcp-client/src/McpClient.ts).

Our `McpClient` class has
- an InferenceClient (works with any inference provider, and `huggingface/inference` supports both remote and local endpoints)
- a set of MCP client sessions, one of each connected MCP server (this allows us to connect to multiple servers, including Gradio server)
- a list of available tools that is going to be filled from the connected servers and slightly re-formatted

```typescript
export class McpClient {
	protected client: InferenceClient;
	protected provider: string;
	protected model: string;
	private clients: Map<ToolName, Client> = new Map();
	public readonly availableTools: ChatCompletionInputTool[] = [];

	constructor({ provider, model, apiKey }: { provider: InferenceProvider; model: string; apiKey: string }) {
		this.client = new InferenceClient(apiKey);
		this.provider = provider;
		this.model = model;
	}

	// [...]
}
```


To connect to a MCP server (like our Gradio sentiment analysis server), the official TypeScript SDK provides a `Client` class with a `listTools()` method:
```typescript
async addMcpServer(server: StdioServerParameters): Promise<void> {
	const transport = new StdioClientTransport({
		...server,
		env: { ...server.env, PATH: process.env.PATH ?? "" },
	});
	const mcp = new Client({ name: "@huggingface/mcp-client", version: packageVersion });
	await mcp.connect(transport);

	const toolsResult = await mcp.listTools();
	debug(
		"Connected to server with tools:",
		toolsResult.tools.map(({ name }) => name)
	);

	for (const tool of toolsResult.tools) {
		this.clients.set(tool.name, mcp);
	}

	this.availableTools.push(
		...toolsResult.tools.map((tool) => {
			return {
				type: "function",
				function: {
					name: tool.name,
					description: tool.description,
					parameters: tool.inputSchema,
				},
			} satisfies ChatCompletionInputTool;
		})
	);
}
```

The `StdioServerParameters` is an interface from the MCP SDK that will let us easily spawn a local process. For each MCP server we connect to (including our Gradio sentiment analysis server), we slightly re-format its list of tools and add them to `this.availableTools` variable.

Then we pass `this.availableTools` to our LLM chat-completion, in addition to our usual array of messages:
```typescript
const stream = this.client.chatCompletionStream({
	provider: this.provider,
	model: this.model,
	messages,
	tools: this.availableTools,
	tool_choice: "auto",
});
```

`tool_choice: "auto"` is the parameter we pass for the LLM to generate zero, one, or multiple tool calls.

When parsing or streaming the output, the LLM will generate some tool calls (i.e., a function name, and some JSON-encoded arguments), which we need to compute. The MCP client SDK makes it easy by calling the `client.callTool()` method:
```typescript
const toolName = toolCall.function.name;
const toolArgs = JSON.parse(toolCall.function.arguments);

const toolMessage: ChatCompletionInputMessageTool = {
	role: "tool",
	tool_call_id: toolCall.id,
	content: "",
	name: toolName,
};

/// Get the appropriate session for this tool
const client = this.clients.get(toolName);
if (client) {
	const result = await client.callTool({ name: toolName, arguments: toolArgs });
	toolMessage.content = result.content[0].text;
} else {
	toolMessage.content = `Error: No session found for tool: ${toolName}`;
}
```

If the LLM chooses to use our sentiment analysis tool, this code will automatically route the call to our Gradio server, execute the analysis, and return the result back to the LLM. Finally we will add the resulting tool message to our `messages` array and back into the LLM.